# Trabalho prático da disciplina Mineração de Dados

## Alunos:
- ### Pedro Augusto Sousa Gonçalves, 21.1.4015
- ### Tiago Gomes da Silva - 19.2.4009

#### Competição: https://www.kaggle.com/competitions/march-machine-learning-mania-2025
#### Repositorio: https://github.com/pedroasgDEV/NCAA_DataMine

Importando os arquivos de dados

### 1. Introdução

O torneio de basquete universitário dos Estados Unidos, conhecido como NCAA, é uma competição de eliminação direta com altíssimo grau de imprevisibilidade, onde dezenas de universidades competem por meio de confrontos intensos, muitas vezes decididos por margens estreitas. A competição "March Machine Learning Mania 2025" do Kaggle propõe um problema de mineração de dados que visa aplicar técnicas de aprendizado de máquina para prever os resultados dos jogos do torneio masculino e feminino, utilizando dados históricos e estatísticas das equipes.

#### 1.1 Motivação

A aplicação da mineração de dados em esportes, especialmente no torneio NCAA, apresenta um cenário desafiador devido à complexidade do domínio, marcada por fatores como variações de desempenho, estilos de jogo, lesões, formato de eliminação direta e aleatoriedade. Dessa forma, trata-se de um contexto ideal para testar e aplicar as técnicas apresentadas ao longo do curso.

#### 1.2 Objetivo do Trabalho

O objetivo principal é desenvolver modelos preditivos capazes de estimar a probabilidade de vitória de todos os possíveis confrontos do torneio masculino da NCAA de 2025. Devido ao caráter eliminatório direto, o problema exige que todas as possibilidades sejam consideradas, mesmo aqueles confrontos que nunca irão ocorrer. As previsões devem ser probabilísticas, refletindo a incerteza inerente a cada possível confronto.

### 2. Materiais e Métodos

#### 2.1 Bibliotecas Utilizadas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk

#### 2.2 Dados usados

In [13]:
# Dados Gerais do NCCA
general_data_path = "march-machine-learning-mania-2025/"
cities = pd.read_csv(general_data_path + "Cities.csv")
conferences = pd.read_csv(general_data_path + "Conferences.csv")

# Dados específicos do MNCAA
MNCAA_data_path = "march-machine-learning-mania-2025/MNCAA/"
conferenceTourneyGames = pd.read_csv(data_path + 'MConferenceTourneyGames.csv')
gameCities = pd.read_csv(data_path + 'MGameCities.csv')
masseyOrdinals = pd.read_csv(data_path + 'MMasseyOrdinals.csv')
tourneyCompactResults = pd.read_csv(data_path + 'MNCAATourneyCompactResults.csv')
tourneyDetailedResults = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv')
tourneySlots = pd.read_csv(data_path + 'MNCAATourneySlots.csv')
regularSeasonCompactResults = pd.read_csv(data_path + 'MRegularSeasonCompactResults.csv')
regularSeasonDetailedResults = pd.read_csv(data_path + 'MRegularSeasonDetailedResults.csv')
seasons = pd.read_csv(data_path + 'MSeasons.csv')
secondaryTourneyCompactResults = pd.read_csv(data_path + 'MSecondaryTourneyCompactResults.csv')
secondaryTourneyTeams = pd.read_csv(data_path + 'MSecondaryTourneyTeams.csv')
teamCoaches = pd.read_csv(data_path + 'MTeamCoaches.csv')
teamConferences = pd.read_csv(data_path + 'MTeamConferences.csv')
teams = pd.read_csv(data_path + 'MTeams.csv')
teamSpellings = pd.read_csv(data_path + 'MTeamSpellings.csv')

##### 2.2.1 Cidades

O arquivo *Cities.csv* armazena todas as 503 cidades onde ocorrem os jogos e os seus respectivos estados.

In [18]:
cities.count()

CityID    503
City      503
State     503
dtype: int64